In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
def noop(*args, **kwargs): pass
warnings.warn = noop

In [3]:
import copy
from collections import ChainMap
from multiprocessing import cpu_count

In [4]:
from IPython.display import display
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

In [5]:
from basedir import SAMPLE
from utils import from_feather, to_feather

In [6]:
seed = 1
np.random.seed(seed)

In [7]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [8]:
class Classifier(nn.Module):
    def __init__(self, n_in, conv=(32, 64, 128, 256), fc=(512, 512), n_out=9):
        super().__init__()
        layers = []
        fi = n_in
        for size in conv:
            layers += [
                nn.Conv1d(fi, size, 3),
                nn.BatchNorm1d(size),
                nn.LeakyReLU(0.05)]
            fi = size
        layers.append(nn.AdaptiveAvgPool1d(1))
        layers.append(Flatten())
        for size in fc:
            layers += [
                nn.Linear(fi, size),
                nn.BatchNorm1d(size),
                nn.LeakyReLU(0.05),
                nn.Dropout(0.15)]
            fi = size
        layers.append(nn.Linear(fi, n_out))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [9]:
def accuracy(y_true, y_pred):
    n = len(y_true)
    y_hat = y_pred.reshape(9, n).argmax(axis=0)
    value = (y_true == y_hat).mean()
    return 'accuracy', value, True

In [10]:
from torch.optim.lr_scheduler import _LRScheduler

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [11]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [12]:
ID_COLS = ['series_id', 'measurement_number']

In [13]:
def create_datasets(X, y, test_size=0.2, dropcols=ID_COLS):
    enc = LabelEncoder()
    y_enc = enc.fit_transform(y)
    X_grouped = np.row_stack([
        group.drop(columns=dropcols).values[None]
        for _, group in X.groupby('series_id')])
    X_grouped = X_grouped.transpose(0, 2, 1)
    X_train, X_valid, y_train, y_valid = train_test_split(X_grouped, y_enc, test_size=0.1)
    X_train, X_valid = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_valid)]
    y_train, y_valid = [torch.tensor(arr, dtype=torch.long) for arr in (y_train, y_valid)]
    train_ds = TensorDataset(X_train, y_train)
    valid_ds = TensorDataset(X_valid, y_valid)
    return train_ds, valid_ds, enc

In [14]:
def create_test_dataset(X, dropcols=ID_COLS):
    X_grouped = np.row_stack([
        group.drop(columns=dropcols).values[None]
        for _, group in X.groupby('series_id')])
    X_grouped = torch.tensor(X_grouped.transpose(0, 2, 1)).float()
    y_fake = torch.tensor([0] * len(X_grouped)).long()
    return TensorDataset(X_grouped, y_fake)

In [15]:
def create_loaders(train_ds, valid_ds, bs=512, jobs=0):
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
    valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)
    return train_dl, valid_dl

In [16]:
def accuracy(output, target):
    return (output.argmax(dim=1) == target).float().mean().item()

In [17]:
x_trn, y_trn, x_tst = from_feather('x_trn', 'y_trn', 'x_tst')

In [20]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [ ]:
lr = 1e-2
wd = 1e-5
bs = 400
n_epochs = 1000
patience = 200
no_improvements = 0
jobs = 12
best_loss = np.inf
best_acc = 0
best_weights = None
history = []
lr_history = []

trn_ds, val_ds, enc = create_datasets(x_trn, y_trn['surface'])
trn_dl, val_dl = create_loaders(trn_ds, val_ds, bs, jobs=jobs)
dataset_sizes = {'train': len(trn_ds), 'val': len(val_ds)}

model = Classifier(10)
model.to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = len(trn_dl)
scheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase, loader in (('train', trn_dl), ('val', val_dl)):
        training = phase == 'train'
        running_loss = 0.0
        num_correct = 0
        num_examples = 0
        n_batches = 0
        
        for batch in loader:
            x_batch, y_batch = [b.to(device) for b in batch]
            optimizer.zero_grad()
        
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                
                # don't update weights and rates when in 'val' phase
                if training:
                    scheduler.step()
                    loss.backward()
                    optimizer.step()
                    lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            num_correct += (outputs.argmax(dim=1) == y_batch).sum().item()
            num_examples += len(x_batch)
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            val_acc = num_correct/num_examples
            # if epoch_loss < best_loss:
            if val_acc > best_acc:
                # print('loss improvement on epoch: %d' % (epoch + 1))
                print('accuracy improvement on epoch: %d' % (epoch + 1))
                best_acc = val_acc
                best_loss = epoch_loss
                best_weights = copy.deepcopy(model.state_dict())
                torch.save(best_weights, 'best_weights.pth')
                no_improvements = 0
            else:
                no_improvements += 1
            stats['val_acc'] = val_acc
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] '
          'train: {train:.4f} - '
          'val: {val:.4f} - '
          'val. acc: {val_acc:2.2%}'.format(**stats))
    
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break

if best_weights is not None:
    # print(f'Loading the best weights with the training loss: {best_loss:.4f}')
    print(f'Loading the best weights with the validation accuracy: {best_acc:2.2%}')
    model.load_state_dict(best_weights)

In [ ]:
model = Classifier(10).to(device)
model.load_state_dict(torch.load('best_weights.pth'))

In [ ]:
test_ds = create_test_dataset(x_tst)

In [ ]:
# _, _, enc = create_datasets(x_trn, y_trn['surface'])

In [ ]:
test_results = []
for x_batch, _ in DataLoader(test_ds, batch_size=1000, shuffle=False):
    output = model(x_batch.to(device))
    test_results += output.argmax(dim=1).tolist()

In [ ]:
submit = pd.read_csv(SAMPLE)
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit.csv', index=None)
!kaggle c submit career-con-2019 -f 'submit.csv' -m "Conv1d first try"